In [1]:
import fastplotlib as fpl
import os
import sys
import masknmf
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%matplotlib inline

fastplotlib version from git (0.5.0) and __version__ (0.6.0) don't match.
Unable to find extension: VK_EXT_acquire_drm_display
Unable to find extension: VK_EXT_physical_device_drm
No config found!
EGL says it can present to the window but not natively
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Valid,Device,Type,Backend,Driver
✅ (default),NVIDIA TITAN RTX,DiscreteGPU,Vulkan,555.42.02
❗ limited,"llvmpipe (LLVM 12.0.0, 256 bits)",CPU,Vulkan,Mesa 21.2.6 (LLVM 12.0.0)
❌,NVIDIA TITAN RTX/PCIe/SSE2,Unknown,OpenGL,3.3.0 NVIDIA 555.42.02


Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


In [2]:
data = tifffile.imread("../../demo_data/demoMovie.tif")

# Define a rigid motion correction strategy. Optionally provide a template as a 2D torch.Tensor

In [3]:
max_rigid_shifts = [3, 3]
template = None
rigid_strategy = masknmf.RigidMotionCorrector(max_rigid_shifts, template = template)

# Run the template estimator

In [10]:
rigid_strategy.compute_template(data)

# Define a motion correction array, which lazily loads motion corrected frames aligned via the registation_strategy

In [7]:
moco_results = masknmf.RegistrationArray(data,
                                         rigid_strategy)

# How to access shifts (and corresponding motion corrected frames). 

In [8]:
moco_shifts = moco_results.shifts[0:500] #Shift data for first 500 frames

In [11]:
fig, ax = plt.subplots(1, 2)
ax[0].plot(moco_shifts[:, 0])
ax[0].set_title("vertical shifts")
ax[1].plot(moco_shifts[:, 1])
ax[1].set_title("horizontal shifts")
plt.show()

# Visualize the raw vs. registered data via fastplotlib

In [11]:
iw = fpl.ImageWidget(data = [data, moco_results, rigid_strategy.template],
                    names = ['raw', 'motion corrected', 'template'],
                    figure_shape = (1, 3))
iw.cmap = "gray"
iw.show()